In [ ]:
from numpy import floating, bool_
from numpy.typing import NDArray

from degradation_eda.processing_stages import load_data
from degradation_eda.utils.visualization import show_frame

In [ ]:
frames: NDArray[floating] = load_data("","")
mask: NDArray[bool_] = load_data("","")
show_frame[frames[0]]